# Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image  as img
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
import keras.backend as K
from tqdm import tqdm
import sys
sys.path.append('../input/swintransformertf')
from swintransformer import SwinTransformer
from sklearn.model_selection import StratifiedKFold

# Data

In [2]:
train_csv=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test_csv=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [3]:
train_csv.head(2)

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42


In [4]:
test_csv.head(2)

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0


In [5]:
labels=dict(train_csv[['Id','Pawpularity']].values)

# Data Generators

In [6]:
class DataGenerator(tf.keras.utils.Sequence):

    def __init__(self, list_IDs, labels, batch_size=18, n_channels=1,part='train', shuffle=False):
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.part=part
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        list_IDs_temp      = [self.list_IDs[k] for k in indexes]
        
        x, y = self.__data_generation(list_IDs_temp)
        return x, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        x           = np.zeros((self.batch_size,384,384,3))
        y           = np.zeros((self.batch_size,1))
        for i, ID in enumerate(list_IDs_temp):
            x[i]                =Image.open('../input/petfinder-pawpularity-score/'+str(self.part)+'/'+ str(ID)+'.jpg').resize((384,384))
            y[i]                =self.labels[ID]

        return x,y

In [7]:
class TestDataGenerator1(tf.keras.utils.Sequence):

    def __init__(self, list_IDs, batch_size=1, n_channels=1,part='test', shuffle=False):
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.part=part
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        list_IDs_temp      = [self.list_IDs[k] for k in indexes]
        
        x= self.__data_generation(list_IDs_temp)
        return x

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        x           = np.zeros((self.batch_size,384,384,3))
        for i, ID in enumerate(list_IDs_temp):
            x[i]                =Image.open('../input/petfinder-pawpularity-score/'+str(self.part)+'/'+ str(ID)+'.jpg').resize((384,384))

        return x

test_generator1   = TestDataGenerator1(test_csv['Id'].values)
test_generator11  = TestDataGenerator1(train_csv['Id'].values)


class TestDataGenerator2(tf.keras.utils.Sequence):

    def __init__(self, list_IDs, batch_size=1, n_channels=1,part='test', shuffle=False):
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.part=part
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        list_IDs_temp      = [self.list_IDs[k] for k in indexes]
        
        x= self.__data_generation(list_IDs_temp)
        return x

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        x           = np.zeros((self.batch_size,224,224,3))
        for i, ID in enumerate(list_IDs_temp):
            x[i]                =Image.open('../input/petfinder-pawpularity-score/'+str(self.part)+'/'+ str(ID)+'.jpg').resize((224,224))

        return x

test_generator2   = TestDataGenerator2(test_csv['Id'].values)
test_generator21  = TestDataGenerator2(train_csv['Id'].values)

# CallBack

In [8]:
class CustomCallBacks(tf.keras.callbacks.Callback):
    
    def __init__(self,n):
        self.n=n
    
    def on_train_begin(self, logs={}):
        self.loss     ={'loss':[]}
        self.val_loss ={'loss':[]}
        self.epoch    =None
        self.b        =0
        self.c        =0
        self.min_t    =None 
        
    def on_epoch_begin(self, epoch, logs={}):
        self.epoch=epoch
        if self.epoch>=1 and self.epoch%2==0 :
            lr=tf.keras.backend.get_value(self.model.optimizer.lr)
            tf.keras.backend.set_value(self.model.optimizer.lr,lr*0.50)

    def on_epoch_end(self, epoch, logs={}):
        print('{} ended'.format(epoch))
        self.loss['loss'].append(logs.get('loss'))
        self.val_loss['loss'].append(logs.get('val_loss'))
        if self.epoch==0:
            self.model.save_weights('model'+str(self.n)+'.h5')
            self.min_t=self.val_loss['loss'][-1]
        if self.epoch>=1:
            if self.val_loss['loss'][-1]<=self.min_t:
                self.min_t=self.val_loss['loss'][-1]
                self.model.save_weights('model'+str(self.n)+'.h5')
                print('*'*5)
                print("weight updated")
                print('*'*5)

# Model

In [9]:
model0 = tf.keras.Sequential([
      tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode="torch"), input_shape=(224,224,3)),
      SwinTransformer('swin_large_224', include_top=False, pretrained=False),
      tf.keras.layers.Dropout(0.01),
      tf.keras.layers.Dense(1, activation='linear')
    ])
model0.layers[-3].trainable=False
model0.load_weights('../input/p-pr-trail-swin-overfit-01/model0.h5')


model1 = tf.keras.Sequential([
      tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode="torch"), input_shape=(224,224,3)),
      SwinTransformer('swin_large_224', include_top=False, pretrained=False),
      tf.keras.layers.Dropout(0.01),
      tf.keras.layers.Dense(1, activation='linear')
    ])
model1.layers[-3].trainable=False
model1.load_weights('../input/p-pr-trail-swin-overfit-01/model1.h5')

model2 = tf.keras.Sequential([
      tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode="torch"), input_shape=(224,224,3)),
      SwinTransformer('swin_large_224', include_top=False, pretrained=False),
      tf.keras.layers.Dropout(0.01),  
      tf.keras.layers.Dense(1, activation='linear')
    ])
model2.layers[-3].trainable=False
model2.load_weights('../input/p-pr-trail-swin-overfit-01/model2.h5')

model3 = tf.keras.Sequential([
      tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode="torch"), input_shape=(224,224,3)),
      SwinTransformer('swin_large_224', include_top=False, pretrained=False),
      tf.keras.layers.Dropout(0.01),  
      tf.keras.layers.Dense(1, activation='linear')
    ])
model3.layers[-3].trainable=False
model3.load_weights('../input/p-pr-trail-swin-overfit-01/model3.h5')

model4 = tf.keras.Sequential([
      tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode="torch"), input_shape=(224,224,3)),
      SwinTransformer('swin_large_224', include_top=False, pretrained=False),
      tf.keras.layers.Dropout(0.01),
      tf.keras.layers.Dense(1, activation='linear')
    ])
model4.layers[-3].trainable=False
model4.load_weights('../input/p-pr-trail-swin-overfit-01/model4.h5')

model5 = tf.keras.Sequential([
      tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode="torch"), input_shape=(224,224,3)),
      SwinTransformer('swin_large_224', include_top=False, pretrained=False),
      tf.keras.layers.Dropout(0.01),  
      tf.keras.layers.Dense(1, activation='linear')
    ])
model5.layers[-3].trainable=False
model5.load_weights('../input/p-pr-trail-swin-overfit-01/model5.h5')


model6 = tf.keras.Sequential([
      tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode="torch"), input_shape=(224,224,3)),
      SwinTransformer('swin_large_224', include_top=False, pretrained=False),
      tf.keras.layers.Dropout(0.01),
      tf.keras.layers.Dense(1, activation='linear')
    ])
model6.layers[-3].trainable=False
model6.load_weights('../input/p-pr-trail-swin-overfit-01/model6.h5')

model7 = tf.keras.Sequential([
      tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode="torch"), input_shape=(224,224,3)),
      SwinTransformer('swin_large_224', include_top=False, pretrained=False),
      tf.keras.layers.Dropout(0.01),
      tf.keras.layers.Dense(1, activation='linear')
    ])
model7.layers[-3].trainable=False
model7.load_weights('../input/p-pr-trail-swin-overfit-01/model7.h5')

model8 = tf.keras.Sequential([
      tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode="torch"), input_shape=(224,224,3)),
      SwinTransformer('swin_large_224', include_top=False, pretrained=False),
      tf.keras.layers.Dropout(0.01),
      tf.keras.layers.Dense(1, activation='linear')
    ])
model8.layers[-3].trainable=False
model8.load_weights('../input/p-pr-trail-swin-overfit-01/model8.h5')

model9 = tf.keras.Sequential([
      tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode="torch"), input_shape=(224,224,3)),
      SwinTransformer('swin_large_224', include_top=False, pretrained=False),
      tf.keras.layers.Dropout(0.01),
      tf.keras.layers.Dense(1, activation='linear')
    ])
model9.layers[-3].trainable=False
model9.load_weights('../input/p-pr-trail-swin-overfit-01/model9.h5')

2022-01-10 00:15:11.686089: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-10 00:15:11.790021: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-10 00:15:11.791066: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-10 00:15:11.792612: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [10]:
y_test0=model0.predict_generator(test_generator2,verbose=1)
y_test1=model1.predict_generator(test_generator2,verbose=1)
y_test2=model2.predict_generator(test_generator2,verbose=1)
y_test3=model3.predict_generator(test_generator2,verbose=1)
y_test4=model4.predict_generator(test_generator2,verbose=1)
y_test5=model5.predict_generator(test_generator2,verbose=1)
y_test6=model6.predict_generator(test_generator2,verbose=1)
y_test7=model7.predict_generator(test_generator2,verbose=1)
y_test8=model8.predict_generator(test_generator2,verbose=1)
y_test9=model9.predict_generator(test_generator2,verbose=1)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
2022-01-10 00:19:09.350502: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-10 00:19:16.025989: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


8/8 [==============================] - 6s 85ms/step


In [11]:
test_csv['Pawpularity']=(y_test1+y_test2+y_test3+y_test4+y_test0+y_test5+y_test6+y_test7+y_test8+y_test9)/10

In [12]:
test_csv[['Id','Pawpularity']].to_csv('submission.csv',index=False)